# Images Clean Up

Remove all the pictures with width less then 250.

In [1]:
import sys 
import os 
sys.path.append(os.getcwd()) 
import cv2
import numpy as np 
from tqdm import tqdm 
import dlib

In [2]:
def preprocess(image_path):

    img = cv2.imread(image_path)
    w, h, c = img.shape
    if w < 250 or h < 250:
        os.remove(image_path)
        return 0
    return 1


In [4]:
data_path = 'Datasets//train'
files_list = os.listdir(data_path)
# Loop through all the files
kill_img = 0
not_kill = 0
for man in tqdm(files_list):
    pic_list = os.listdir(os.path.join(data_path, man))
    for pic in pic_list:
        img_path = os.path.join(data_path, man, pic)
        a = preprocess(img_path)
        if a == 0:
            kill_img += 1
        elif a == 1:
            not_kill += 1
print('kill:', kill_img)
print('not_kill:', not_kill)
print('all:', kill_img+not_kill)

100%|██████████| 8631/8631 [3:46:36<00:00,  1.58s/it]  

kill: 1979160
not_kill: 1162730
all: 3141890


# Generate Data List For Training
Store all the data files information into a csv file

In [1]:
import sys
import os
sys.path.append(os.getcwd())
from tqdm import tqdm
import csv 

In [2]:
# Define file path
data_path = 'Datasets//train_face_mask'
csv_path = 'train_face_mask.csv'

# Open a csv file
f = open(csv_path,'w',encoding='utf-8')
csv_writer = csv.writer(f)
# Enter column title
csv_writer.writerow(["id","name","class"])

files_list = os.listdir(data_path)
files_list.sort()
# Loop through the files
for index, man in tqdm(enumerate(files_list)):
    pic_list = os.listdir(os.path.join(data_path, man))
    pic_list.sort()
    for pic in pic_list:
        picid = pic.split('.')
        csv_writer.writerow([picid[0],man,index])

f.close()

948it [00:52, 18.00it/s]  


In [3]:
# Define file path
data_path = 'Datasets//train_face_notmask'
csv_path = 'train_face_notmask.csv'
# Open a csv file
f = open(csv_path, 'w', encoding='utf-8', newline='')
csv_writer = csv.writer(f)
# Enter column title
csv_writer.writerow(["id","name","class"])

files_list = os.listdir(data_path)
files_list.sort()
# Loop through the files
for index, man in tqdm(enumerate(files_list)):
    pic_list = os.listdir(os.path.join(data_path, man))
    pic_list.sort()
    for pic in pic_list:
        picid = pic.split('.')
        csv_writer.writerow([picid[0],man,index])

f.close()

948it [02:10,  7.29it/s]


# Cut Images/ Alignment/ Mask Generation
This process is based on the 68 face landmarks model in the OpenCV library

In [3]:
import sys
import copy
import os 
sys.path.append(os.getcwd()) 
from tqdm import tqdm 
import numpy as np 
import dlib 
import cv2
import json
import argparse

In [4]:
parser = argparse.ArgumentParser(description='manual to this script')
parser.add_argument('-s1', '--segment1',type=int, default = 0)
parser.add_argument('-s2', '--segment2', type=int, default=10)
parser.add_argument('-sa', '--segmentall', type=int, default=10)
args, unknown = parser.parse_known_args()
if (args.segmentall<args.segment2) or (args.segment1>=args.segment2) or (args.segmentall<args.segment1):
    print('Segment setting is wrong！')
    sys.exit(-1)

os.environ["CUDA_VISIBLE_DEVICES"] = "3"

class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(MyEncoder, self).default(obj)

pwd = os.path.abspath('../')

print('dlib.DLIB_USE_CUDA:', dlib.DLIB_USE_CUDA)
print('dlib.cuda.get_num_devices():', dlib.cuda.get_num_devices())

dlib.DLIB_USE_CUDA: False
dlib.cuda.get_num_devices(): 1


In [19]:
def preprocess(image_path, saveimg_path, savemask_path, saveimg_path_notmask, savemask_path_notmask, \
               detector, predictor, img_size, masked, notmasked):
    global jsonfile
    image = dlib.load_rgb_image(image_path)
    face_img, mask_img = None, None
    # Face cut and align
    dets = detector(image, 1)
    name, id_ = image_path.split(os.sep)[-2:]
    id_ = id_.split('.')[0]
    xmin = None
    xmax = None
    ymin = None
    ymax = None
    if len(dets) == 1:
        faces = dlib.full_object_detections()
        faces.append(predictor(image, dets[0]))
        images = dlib.get_face_chips(image, faces, size=img_size)

        image = np.array(images[0]).astype(np.uint8)
        face_img = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        origin_face_img = copy.deepcopy(face_img)
        # Generate mask
        dets = detector(image, 1)
        if len(dets) == 1:
            point68 = predictor(image, dets[0])
            landmarks = list()
            INDEX = [0,2,14,16,17,18,19,24,25,26]
            eyebrow_list = [19, 24]
            eyes_list = [36, 45]
            eyebrow = 0
            eyes = 0

            for eb, ey in zip(eyebrow_list, eyes_list):
                eyebrow += point68.part(eb).y
                eyes += point68.part(ey).y
            add_pixel = int(eyes/2 - eyebrow/2)

            for idx in INDEX:
                x = point68.part(idx).x
                if idx in eyebrow_list:
                    y = (point68.part(idx).y - 2*add_pixel) if (point68.part(idx).y - 2*add_pixel) > 0 else 0
                else:
                    y = point68.part(idx).y
                landmarks.append((x, y))
        
            belows = []
            for i in range(2, 15, 1):
                belows.append([point68.part(i).x, point68.part(i).y])
            belows = np.array(belows)
            colors = [(200, 183, 144), (163, 150, 134), (172, 170, 169), \
                      (167, 168, 166), (173, 171, 170), (161, 161, 160), \
                      (170, 162, 162)]
            cl = np.random.choice(len(colors), 1)[0]
            cv2.fillConvexPoly(face_img, belows, colors[cl])

            landmarks = np.array(landmarks)
            hull = cv2.convexHull(landmarks)
            mask = np.zeros(origin_face_img.shape, dtype=np.uint8)
            mask_img = cv2.fillPoly(mask, [hull], [255, 255, 255])

            lm = np.array(landmarks)
            h, w, c = face_img.shape
            xmin = int(max(0, np.min(lm[:, 0])))
            xmax = int(min(np.max(lm[:, 0]), w))

            ymax = int(min(np.max(lm[:, 1]), h))
    # if np.max(face_img) is not None and np.max(mask_img) is not None:
    if np.max(face_img) is not None and xmin is not None:
        if notmasked:
            cv2.imwrite(savemask_path_notmask, mask_img)
        if masked:
            with open(savemask_path, 'w') as obj:
                obj.write(str(xmin) + ',0,'+str(xmax)+','+str(ymax))
                obj.write('\n')
        if masked:
            cv2.imwrite(saveimg_path, face_img)
        if notmasked:
            cv2.imwrite(saveimg_path_notmask, origin_face_img)
  

In [22]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
img_size = 256

data_path = os.path.join(pwd, f'data{os.sep}Datasets{os.sep}train')
face_path_mask = os.path.join(pwd, f'data{os.sep}Datasets{os.sep}train_face_mask')
face_path_notmask = os.path.join(pwd, f'data{os.sep}Datasets{os.sep}train_face_notmask')
mask_path_mask = os.path.join(pwd, f'data{os.sep}Datasets{os.sep}train_mask_mask')
mask_path_notmask = os.path.join(pwd, f'data{os.sep}Datasets{os.sep}train_mask_notmask')
if not os.path.exists(face_path_mask):
    os.mkdir(face_path_mask)
if not os.path.exists(mask_path_mask):
    os.mkdir(mask_path_mask)
if not os.path.exists(face_path_notmask):
    os.mkdir(face_path_notmask)
if not os.path.exists(mask_path_notmask):
    os.mkdir(mask_path_notmask)

files_list = os.listdir(data_path)
length = len(files_list)
files_list = files_list[(length*args.segment1//args.segmentall):(length*args.segment2//args.segmentall)]

def read_json(jsonfile):
    with open(jsonfile, 'r', encoding = 'utf-8') as f:
        file = json.load(f)
    return file

#Whether to generate masked pictures with related mask
masked = True

#Whether to generate not masked pictures with related mask
notmasked = True

for man in tqdm(files_list):
    pic_list = os.listdir(os.path.join(data_path, man))
    if masked:
        if not os.path.exists(os.path.join(mask_path_mask, man)):
            os.mkdir(os.path.join(mask_path_mask, man))
        if not os.path.exists(os.path.join(face_path_mask, man)):
            os.mkdir(os.path.join(face_path_mask, man))
    if notmasked:
        if not os.path.exists(os.path.join(mask_path_notmask, man)):
            os.mkdir(os.path.join(mask_path_notmask, man))
        if not os.path.exists(os.path.join(face_path_notmask, man)):
            os.mkdir(os.path.join(face_path_notmask, man))

    for pic in pic_list:
        img_path = os.path.join(data_path, man, pic)
        save_txt_path_mask = os.path.join(mask_path_mask, man, pic.replace('.jpg', '.txt').replace('.png', '.txt'))
        save_mask_path_notmask = os.path.join(mask_path_notmask, man, pic)
        save_face_path_mask = os.path.join(face_path_mask, man, pic)
        save_face_path_notmask = os.path.join(face_path_notmask, man, pic)
        preprocess(img_path, save_face_path_mask, save_txt_path_mask, save_face_path_notmask, \
                   save_mask_path_notmask, detector, predictor, img_size,masked, notmasked)



100%|██████████| 8631/8631 [40:04:36<00:00, 16.72s/it]   


# Training

In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
import sys
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

sys.path.append(os.getcwd())

from torch.nn.modules.distance import PairwiseDistance
import torch.nn as nn
from tqdm import tqdm
import numpy as np
import time

from Data_loader.Data_loader_facenet_mask import test_dataloader,V9_train_dataloader
from Data_loader.Data_loader_facenet_mask import LFWestMask_dataloader
from Losses.Triplet_loss import TripletLoss
from validate_on_LFW import evaluate_lfw
from Data_loader.Data_loader_train_notmask import TrainDataset

from config_mask import config
from Models.CBAM_Face_attention_Resnet_notmaskV3 import resnet18_cbam, resnet50_cbam, resnet101_cbam, resnet34_cbam, \
    resnet152_cbam


C:\Users\jtyan\AppData\Roaming\Python\Python39\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")



load 12500 triplets...
12500 triplets loaded!

load lfw_paths...
lfw_paths loaded!

load lfw_paths...
lfw_paths loaded!

load 12500 triplets...
12500 triplets loaded!


In [3]:
pwd = os.path.abspath('./')
print(torch.cuda.is_available())
print("Using {} model architecture.".format(config['model']))
start_epoch = 0

# Model 34 is picked for this training
if config['model'] == 18:
    model = resnet18_cbam(pretrained=True, showlayer= False,num_classes=128)
elif config['model'] == 34:
    model = resnet34_cbam(pretrained=True, showlayer= False, num_classes=128)
elif config['model'] == 50:
    model = resnet50_cbam(pretrained=True, showlayer= False, num_classes=128)
elif config['model'] == 101:
    model = resnet101_cbam(pretrained=True, showlayer= False, num_classes=128)
elif config['model'] == 152:
    model = resnet152_cbam(pretrained=True, showlayer= False, num_classes=128)

model_path = os.path.join(pwd, 'Model_training_checkpoints')
x = [int(i.split('_')[4]) for i in os.listdir(model_path) if 'V9' in i]
x.sort()
for i in os.listdir(model_path):
    if (len(x)!=0) and ('epoch_'+str(x[-1]) in i) and ('V9' in i):
        model_pathi = os.path.join(model_path, i)
        break

if os.path.exists(model_pathi) and ('V9' in model_pathi):
    model_state = torch.load(model_pathi)
    model.load_state_dict(model_state['model_state_dict'])
    start_epoch = model_state['epoch']
    print('loaded %s' % model_pathi)
else:
    print('Training dataset is not exist！')

    

flag_train_gpu = torch.cuda.is_available()
flag_train_multi_gpu = False
if flag_train_gpu and torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)
    model.cuda()
    flag_train_multi_gpu = True
    print('Using multi-gpu training.')
elif flag_train_gpu and torch.cuda.device_count() == 1:
    model.cuda()
    print('Using single-gpu training.')

def adjust_learning_rate(optimizer, epoch):
    if epoch<19:
        lr =  0.125
    elif (epoch>=19) and (epoch<60):
        lr = 0.0625
    elif (epoch >= 60) and (epoch < 90):
        lr = 0.0155
    elif (epoch >= 90) and (epoch < 120):
        lr = 0.003
    elif (epoch>=120) and (epoch<160):
        lr = 0.0001
    else:
        lr = 0.00006
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

# Adam is picked as the optimizer
def create_optimizer(model, new_lr):
    # setup optimizer
    if config['optimizer'] == "sgd":
        optimizer_model = torch.optim.SGD(model.parameters(), lr = new_lr,
                              momentum=0.9, dampening=0.9,
                              weight_decay=0)
    elif config['optimizer'] == "adagrad":
        optimizer_model = torch.optim.Adagrad(model.parameters(), lr = new_lr,
                                  lr_decay=1e-4,
                                  weight_decay=0)

    elif config['optimizer'] == "rmsprop":
        optimizer_model = torch.optim.RMSprop(model.parameters(), lr = new_lr)

    elif config['optimizer'] == "adam":
        optimizer_model = torch.optim.Adam(model.parameters(), lr = new_lr,
                               weight_decay=0)
    return optimizer_model

True
Using 34 model architecture.
loaded D:\UNCC\22_Spring\ITCS 5156 Applied Machine Learning\Slides\Project\data\Model_training_checkpoints\model_34_triplet_epoch_28_rocNotMasked0.789_rocMasked0.663notmaskV9.pt
Using single-gpu training.


In [ ]:
# Random seed
seed = 0
optimizer_model = create_optimizer(model, 0.125)
torch.manual_seed(seed)  # Set seed for CPU
torch.cuda.manual_seed(seed)  # Set seed for current GPU
torch.cuda.manual_seed_all(seed)  # Set seed for all the GPU
np.random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

# epoch
total_time_start = time.time()
start_epoch = start_epoch
end_epoch = start_epoch + config['epochs']
# import l2 calculation
l2_distance = PairwiseDistance(2).cuda()
# For report
best_roc_auc = -1
best_accuracy = -1
print('Countdown 3 seconds')
time.sleep(1)
print('Countdown 2 seconds')
time.sleep(1)
print('Countdown 1 seconds')
time.sleep(1)

# epoch loop
for epoch in range(start_epoch, end_epoch):
    torch.cuda.empty_cache()
    print("\ntraining on TrainDataset! ...")
    epoch_time_start = time.time()
    triplet_loss_sum = 0
    attention_loss_sum = 0
    num_hard = 0

    model.train()  # train the model
    # step loop
    progress_bar = enumerate(tqdm(V9_train_dataloader))
    for batch_idx, (batch_sample) in progress_bar:
        # for batch_idx, (batch_sample) in enumerate(train_dataloader):
        # length = len(train_dataloader)
        # fl=open('/home/Mask-face-recognitionV1/output.txt', 'w')
        # for batch_idx, (batch_sample) in enumerate(train_dataloader):
        # print(batch_idx, end=' ')
        # fl.write(str(batch_idx)+' '+str(round((time.time()-epoch_time_start)*length/((batch_idx+1)*60), 2))+'；  ')
        # Getting the datasets for the iteration
        # Getting three pictures without mask
        anc_img = batch_sample['anc_img'].cuda()
        pos_img = batch_sample['pos_img'].cuda()
        neg_img = batch_sample['neg_img'].cuda()
        # Getting three pictures with mask
        mask_anc = batch_sample['mask_anc'].cuda()
        mask_pos = batch_sample['mask_pos'].cuda()
        mask_neg = batch_sample['mask_neg'].cuda()

        # model calculation
        #  Feedforward- the model is trained by three pictures, embedding and loss is generated. (only two pictures are input with loss during training, and one picture is input with embedding during testing)
        anc_embedding, anc_attention_loss = model((anc_img, mask_anc))
        pos_embedding, pos_attention_loss = model((pos_img, mask_pos))
        neg_embedding, neg_attention_loss = model((neg_img, mask_neg))
        anc_embedding = torch.div(anc_embedding, torch.norm(anc_embedding)) * 50
        pos_embedding = torch.div(pos_embedding, torch.norm(pos_embedding)) * 50
        neg_embedding = torch.div(neg_embedding, torch.norm(neg_embedding)) * 50
        
        # Calculating l2 embedding
        pos_dist = l2_distance.forward(anc_embedding, pos_embedding)
        neg_dist = l2_distance.forward(anc_embedding, neg_embedding)
        # Looking for difficult sample
        all = (neg_dist - pos_dist < config['margin']).cpu().numpy().flatten()
        hard_triplets = np.where(all == 1)
        if len(hard_triplets[0]) == 0:
            continue

        # Select embedding of the difficult samples
        anc_hard_embedding = anc_embedding[hard_triplets].cuda()
        pos_hard_embedding = pos_embedding[hard_triplets].cuda()
        neg_hard_embedding = neg_embedding[hard_triplets].cuda()
        # Select attention loss of the difficult samples
        hard_anc_attention_loss = anc_attention_loss[hard_triplets]
        hard_pos_attention_loss = pos_attention_loss[hard_triplets]
        hard_neg_attention_loss = neg_attention_loss[hard_triplets]

        # loss calculation
        triplet_loss = TripletLoss(margin=config['margin']).forward(
            anchor=anc_hard_embedding,
            positive=pos_hard_embedding,
            negative=neg_hard_embedding
        ).cuda()

        # Calculating the mean for attention loss
        hard_attention_loss = torch.cat([hard_anc_attention_loss, hard_pos_attention_loss, hard_neg_attention_loss])
        # hard_attention_loss = torch.cat([anc_attention_loss, pos_attention_loss, neg_attention_loss])
        hard_attention_loss = torch.mean(hard_attention_loss).cuda()
        hard_attention_loss = hard_attention_loss.type(torch.FloatTensor)
        
        LOSS = triplet_loss + hard_attention_loss

        # Back propagation process
        optimizer_model.zero_grad()
        LOSS.backward()
        optimizer_model.step()

        # update the optimizer learning rate
        adjust_learning_rate(optimizer_model, epoch)

        # Record log info
        # Record the # of difficult sample
        num_hard += len(anc_hard_embedding)
        # Calculate the total loss in this epoch
        triplet_loss_sum += triplet_loss.item()
        attention_loss_sum += hard_attention_loss.item()
       

    # Calculate the average loss in this epoch
    avg_triplet_loss = 0 if (num_hard == 0) else triplet_loss_sum / num_hard
    avg_attention_loss = 0 if (num_hard == 0) else attention_loss_sum / num_hard
    avg_loss = avg_triplet_loss + avg_attention_loss
    epoch_time_end = time.time()

    # Calculate the testing accuracy
    print("Validating on TestDataset! ...")
    model.eval()  # Model evaluation
    with torch.no_grad():
        distances, labels = [], []

        progress_bar = enumerate(tqdm(test_dataloader))
        for batch_index, (data_a, data_b, label) in progress_bar:
            # data_a, data_b, label are in one matrix
            data_a = data_a.cuda()
            data_b = data_b.cuda()
            label = label.cuda()
            output_a, output_b = model(data_a), model(data_b)
            output_a = torch.div(output_a, torch.norm(output_a))
            output_b = torch.div(output_b, torch.norm(output_b))
            distance = l2_distance.forward(output_a, output_b)
            # Matrices in a list
            labels.append(label.cpu().detach().numpy())
            distances.append(distance.cpu().detach().numpy())
        labels = np.array([sublabel for label in labels for sublabel in label])
        distances = np.array([subdist for distance in distances for subdist in distance])
        true_positive_rate, false_positive_rate, precision, recall, accuracy, roc_auc, best_distances, \
        tar, far = evaluate_lfw(
            distances=distances,
            labels=labels,
            epoch = 'epoch_'+str(epoch),
            tag = 'NOTMaskedLFW_auc',
            version = 'V9',
            pltshow=True
        )
    print("Validating on LFWMASKTestDataset! ...")
    with torch.no_grad():  
        distances, labels = [], []
        progress_bar = enumerate(tqdm(LFWestMask_dataloader))
        for batch_index, (data_a, data_b, label) in progress_bar:
            # data_a, data_b, label are in one matrix
            data_a = data_a.cuda()
            data_b = data_b.cuda()
            label = label.cuda()
            output_a, output_b = model(data_a), model(data_b)
            output_a = torch.div(output_a, torch.norm(output_a))
            output_b = torch.div(output_b, torch.norm(output_b))
            distance = l2_distance.forward(output_a, output_b)
            # Matrices in a list
            labels.append(label.cpu().detach().numpy())
            distances.append(distance.cpu().detach().numpy())
        
        labels = np.array([sublabel for label in labels for sublabel in label])
        distances = np.array([subdist for distance in distances for subdist in distance])
        true_positive_rate_mask, false_positive_rate_mask, precision_mask, recall_mask, \
        accuracy_mask, roc_auc_mask, best_distances_mask, \
        tar_mask, far_mask = evaluate_lfw(
            distances=distances,
            labels=labels,
            epoch = 'epoch_'+str(epoch),
            tag = 'MaskedLFW_auc',
            version = 'V9',
            pltshow=True
        )

    # Print and save report
    # Read the best roc and acc value from previous file and update them
    if os.path.exists('logs/lfw_{}_log_tripletnotmaskV9.txt'.format(config['model'])):
        with open('logs/lfw_{}_log_tripletnotmaskV9.txt'.format(config['model']), 'r') as f:
            lines = f.readlines()
            my_line = lines[-3]
            my_line = my_line.split('\t')
            best_roc_auc = float(my_line[3].split(':')[1])
            best_accuracy = float(my_line[5].split(':')[1])

    # Save when the last epoch and auc is the higest
    save = True
    if config['save_last_model'] and epoch == end_epoch - 1:
        save = True
    if roc_auc > best_roc_auc:
        best_roc_auc = roc_auc
        save = True
    if np.mean(accuracy) > best_accuracy:
        best_accuracy = np.mean(accuracy)
    if epoch % 3 == 0:
        save = True
    print('save: ', save)

    # Print the report details
    print('Epoch {}:\n \
           train_log:\tLOSS: {:.3f}\ttri_loss: {:.3f}\tatt_loss: {:.3f}\thard_sample: {}\ttrain_time: {}\n \
           test_log:\tAUC: {:.3f}\tACC: {:.3f}+-{:.3f}\trecall: {:.3f}+-{:.3f}\tPrecision {:.3f}+-{:.3f}\t'.format(
        epoch + 1,
        avg_loss,
        avg_triplet_loss,
        avg_attention_loss,
        num_hard,
        (epoch_time_end - epoch_time_start) / 3600,
        roc_auc,
        np.mean(accuracy),
        np.std(accuracy),
        np.mean(recall),
        np.std(recall),
        np.mean(precision),
        np.std(precision),
    )
    )
    # Print the report for masked data
    print('Epoch {}:\n \
                   train_log:\tLOSS: {:.3f}\ttri_loss: {:.3f}\tatt_loss: {:.3f}\thard_sample: {}\ttrain_time: {}\n \
                   MASKED_LFW_test_log:\tAUC: {:.3f}\tACC: {:.3f}+-{:.3f}\trecall: {:.3f}+-{:.3f}\tPrecision {:.3f}+-{:.3f}\t'.format(
        epoch + 1,
        avg_loss,
        avg_triplet_loss,
        avg_attention_loss,
        num_hard,
        (epoch_time_end - epoch_time_start) / 3600,
        roc_auc_mask,
        np.mean(accuracy_mask),
        np.std(accuracy_mask),
        np.mean(recall_mask),
        np.std(recall_mask),
        np.mean(precision_mask),
        np.std(precision_mask),
    )
    )

    # Save the report as a file
    with open('logs/lfw_{}_log_tripletnotmaskV9.txt'.format(config['model']), 'a') as f:
        val_list = [
            'epoch: ' + str(epoch + 1) + '\t',
            'train:\t',
            'LOSS: ' + str('%.3f' % avg_loss) + '\t',
            'tri_loss: ' + str('%.3f' % avg_triplet_loss) + '\t',
            'att_loss: ' + str('%.3f' % avg_attention_loss) + '\t',
            'hard_sample: ' + str(num_hard) + '\t',
            'train_time: ' + str('%.3f' % ((epoch_time_end - epoch_time_start) / 3600))
        ]
        log = ''.join(str(value) for value in val_list)
        f.writelines(log + '\n')
        val_list = [
            'epoch: ' + str(epoch + 1) + '\t',
            'test:\t',
            'auc_masked: ' + str('%.3f' % roc_auc_mask) + '\t',
            'best_auc_MD: ' + str('%.3f' % best_roc_auc) + '\t',
            'acc_MD: ' + str('%.3f' % np.mean(accuracy_mask)) + '+-' + str('%.3f' % np.std(accuracy_mask)) + '\t',
            'best_acc_MD: ' + str('%.3f' % best_accuracy) + '\t',
            'recall_MD: ' + str('%.3f' % np.mean(recall_mask)) + '+-' + str('%.3f' % np.std(recall_mask)) + '\t',
            'precision_MD: ' + str('%.3f' % np.mean(precision_mask)) + '+-' + str(
                '%.3f' % np.std(precision_mask)) + '\t',
            'best_distances_MD: ' + str('%.3f' % np.mean(best_distances_mask)) + '+-' + str(
                '%.3f' % np.std(best_distances_mask)) + '\t',
            'tar_m: ' + str('%.3f' % np.mean(tar_mask)) + '\t',
        ]
        log = ''.join(str(value) for value in val_list)
        f.writelines(log + '\n')
        val_list = [
            'epoch: ' + str(epoch + 1) + '\t',
            'test:\t',
            'auc: ' + str('%.3f' % roc_auc) + '\t',
            'best_auc: ' + str('%.3f' % best_roc_auc) + '\t',
            'acc: ' + str('%.3f' % np.mean(accuracy)) + '+-' + str('%.3f' % np.std(accuracy)) + '\t',
            'best_acc: ' + str('%.3f' % best_accuracy) + '\t',
            'recall: ' + str('%.3f' % np.mean(recall)) + '+-' + str('%.3f' % np.std(recall)) + '\t',
            'precision: ' + str('%.3f' % np.mean(precision)) + '+-' + str('%.3f' % np.std(precision)) + '\t',
            'best_distances: ' + str('%.3f' % np.mean(best_distances)) + '+-' + str(
                '%.3f' % np.std(best_distances)) + '\t',
            'tar_m: ' + str('%.3f' % np.mean(tar)) + '\t',
        ]
        log = ''.join(str(value) for value in val_list)
        f.writelines(log + '\n')
        val_list = [
            'epoch: ' + str(epoch + 1) + '\t',
            'config:\t',
            'LR: ' + str(config['Learning_rate']) + '\t',
            'optimizer: ' + str(config['optimizer']) + '\t',
            'embedding_dim: ' + str(config['embedding_dim']) + '\t',
            'pretrained: ' + str(config['pretrained']) + '\t',
            'image_size: ' + str(config['image_size'])
        ]
        log = ''.join(str(value) for value in val_list)
        f.writelines(log + '\n' + '\n')

    # Save model weights
    if save:
        state = {
            'epoch': epoch + 1,
            'embedding_dimension': config['embedding_dim'],
            'batch_size_training': config['train_batch_size'],
            'model_state_dict': model.state_dict(),
            'model_architecture': config['model'],
            'optimizer_model_state_dict': optimizer_model.state_dict()
        }
        #
        if flag_train_multi_gpu:
            state['model_state_dict'] = model.module.state_dict()
        # For storing best euclidean distance threshold during LFW validation
        # if flag_validate_lfw:
        # state['best_distance_threshold'] = np.mean(best_distances)
        #
        torch.save(state, 'Model_training_checkpoints/model_{}_triplet_epoch_{}_rocNotMasked{:.3f}_rocMasked{:.3f}notmaskV9.pt'.format(config['model'],
                                                                                                     epoch + 1,
                                                                                                     roc_auc, roc_auc_mask))

# Training loop end
total_time_end = time.time()
total_time_elapsed = total_time_end - total_time_start
print("\nTraining finished: total time elapsed: {:.2f} hours.".format(total_time_elapsed / 3600))


Countdown 3 seconds
Countdown 2 seconds
Countdown 1 seconds

training on TrainDataset! ...


100%|██████████| 2500/2500 [17:20<00:00,  2.40it/s] 


Validating on TestDataset! ...


100%|██████████| 1089/1089 [02:03<00:00,  8.83it/s]


Validating on LFWMASKTestDataset! ...


...

100%|██████████| 2500/2500 [08:20<00:00,  5.00it/s]


Validating on TestDataset! ...


100%|██████████| 1089/1089 [01:44<00:00, 10.40it/s]


Validating on LFWMASKTestDataset! ...


  0%|          | 0/1089 [00:00<?, ?it/s]